In [1]:
from mpi4py import MPI
import gmsh

# Initialize gmsh
gmsh.initialize()
gmsh.model.add("rect_with_partial_interface")

if MPI.COMM_WORLD.rank == 0:
    # --------------------------
    # Parameters
    # --------------------------
    Lx, Ly = 1.0, 1.0
    y_interface = 0.5
    x_start, x_end = 0.35, 0.65
    lc = 0.05          # mesh size
    order = 2          # element order

    # --------------------------
    # 1. Define rectangular domain Ω
    # --------------------------
    p1 = gmsh.model.occ.addPoint(0, 0, 0, lc)
    p2 = gmsh.model.occ.addPoint(Lx, 0, 0, lc)
    p3 = gmsh.model.occ.addPoint(Lx, Ly, 0, lc)
    p4 = gmsh.model.occ.addPoint(0, Ly, 0, lc)
    l1 = gmsh.model.occ.addLine(p1, p2)
    l2 = gmsh.model.occ.addLine(p2, p3)
    l3 = gmsh.model.occ.addLine(p3, p4)
    l4 = gmsh.model.occ.addLine(p4, p1)
    loop = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])
    surf = gmsh.model.occ.addPlaneSurface([loop])

    # --------------------------
    # 2. Add an internal partial horizontal line Γ
    # --------------------------
    p5 = gmsh.model.occ.addPoint(x_start, y_interface, 0, lc)
    p6 = gmsh.model.occ.addPoint(x_end, y_interface, 0, lc)
    l5 = gmsh.model.occ.addLine(p5, p6)

    # --------------------------
    # 3. Fragment the surface with the internal line (split mesh)
    # --------------------------
    gmsh.model.occ.fragment([(2, surf)], [(1, l5)])
    gmsh.model.occ.synchronize()

    # --------------------------
    # 4. Define physical groups
    # --------------------------
    # Surface (bulk Ω)
    gmsh.model.addPhysicalGroup(2, [surf], 1)
    gmsh.model.setPhysicalName(2, 1, "Omega")

    # Identify the internal line (Γ)
    all_lines = gmsh.model.getEntities(dim=1)
    gamma_tag = all_lines[-1][1]  # the last 1D entity created
    gmsh.model.addPhysicalGroup(1, [gamma_tag], 2)
    gmsh.model.setPhysicalName(1, 2, "Gamma")

    # Optionally, tag boundary (outer edges)
    outer_edges = [l1, l2, l3, l4]
    gmsh.model.addPhysicalGroup(1, outer_edges, 3)
    gmsh.model.setPhysicalName(1, 3, "Boundary")

    # --------------------------
    # 5. Generate and save mesh
    # --------------------------
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)
    gmsh.write("horizontal.msh")

gmsh.finalize()


Info    : Meshing 1D...                                                                                                  
Info    : [  0%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 9 (Line)
Info    : Done meshing 1D (Wall 0.000458882s, CPU 0.000316s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0144882s, CPU 0.014698s)
Info    : 515 nodes 1040 elements
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 5 order 2
Info    : [ 20%] Meshing curve 6 order 2
Info    : [ 40%] Meshing curve 7 order 2
Info    : [ 60%] Meshing curve 8 order 2
Info    : [ 70%] Meshing curve 9 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.828213
Info    : Done meshing order 2 (Wall 0.00278855s, CPU 0.00

In [1]:
import dolfinx
print(dolfinx.__version__)

0.10.0


In [5]:
from mpi4py import MPI
from dolfinx import mesh, fem
import ufl
import numpy as np

# ---------------------------------------------------------------------
# 1. Mesh and interface tags (similar to Γ tagging in the workshop)
# ---------------------------------------------------------------------
# Load Gmsh mesh (2D bulk Ω + 1D interface Γ)
from dolfinx.io import gmsh as gmshio
# Read the same .msh
msh, cell_markers, facet_markers = gmshio.read_from_msh("horizontal.msh", MPI.COMM_WORLD, 0, gdim=2)[0:3]


# Extract the 1D interface mesh from the 2D mesh (like igridView)
# interface = mesh.create_submesh(domain, domain.topology.dim - 1, np.arange(domain.topology.index_map(domain.topology.dim - 1).size_local, dtype=np.int32))[0]


Info    : Reading 'horizontal.msh'...
Info    : 12 entities
Info    : 1977 nodes
Info    : 968 elements
Info    : Done reading 'horizontal.msh'


In [6]:
tdim = msh.topology.dim  # 2
fdim = tdim - 1             # 1

# Find all facets tagged as Gamma (tag=2)
omega = msh
gamma_entities = facet_markers.find(2)

# Create a submesh for Γ (same as gamma in the JSDokken example)
gamma, gamma_to_omega = mesh.create_submesh(omega, fdim, gamma_entities)[0:2]

In [7]:
# Define interface tag ID (as defined in Gmsh)
Gamma_tag = 2  # <-- replace with your actual tag ID for the interface Γ

# ---------------------------------------------------------------------
# 2. Function spaces
# ---------------------------------------------------------------------
order = 2
V_m = fem.functionspace(omega, ("Lagrange", order))  # bulk Ω
V_f = fem.functionspace(gamma, ("Lagrange", order))   # interface Γ
V_l = fem.functionspace(gamma, ("Lagrange", order))   # multiplier space on Γ
W = ufl.MixedFunctionSpace(V_m, V_f, V_l)

# ---------------------------------------------------------------------
# 3. Trial and Test functions
# ---------------------------------------------------------------------
phi, psi, mu = ufl.TestFunctions(W)
p_m, p_f, lmbd = ufl.TrialFunctions(W)
# p_m = fem.Function(V_m, name="p_m")
# p_f = fem.Function(V_f, name="p_f")
# lmbd = fem.Function(V_f, name="lmbd")

# ---------------------------------------------------------------------
# 4. Spatial coordinates and given data
# ---------------------------------------------------------------------
x = ufl.SpatialCoordinate(msh)
f_m = fem.Constant(msh, 0.0)
f_f = fem.Constant(msh, 0.0)


In [8]:
# dx = ufl.Measure("dx", domain=omega)
# ds = ufl.Measure("ds", domain=omega, subdomain_data=facet_markers, subdomain_id=Gamma_tag)

dx_m  = ufl.Measure("dx", domain=omega)
ds_m  = ufl.Measure("ds", domain=omega, subdomain_data=facet_markers, subdomain_id=Gamma_tag)
dx_f  = ufl.Measure("dx", domain=gamma)  # 1D integral on the submesh

In [9]:
# ---------------------------------------------------------------------
# 5. Weak formulations
# ---------------------------------------------------------------------

# --- Bulk domain Ω ---------------------------------------------------
a_m0 = ufl.inner(ufl.grad(p_m), ufl.grad(phi)) * dx_m
# Coupling term with interface (approximate; corresponds to avg/trace in DUNE)
# In the Dokken example, coupling to Γ is done via ds(Gamma_tag)
# Here we use the interface integral directly on ds(Gamma_tag)
# (You can later replace lmbd with an actual trace variable or interface function)
a_m1 = -lmbd * phi * ds_m
a_m = a_m0 + a_m1
L_m = f_m * phi * dx_m

# --- Interface Γ -----------------------------------------------------
a_f0 = ufl.inner(ufl.grad(p_f), ufl.grad(psi)) * dx_f
a_f1 = lmbd * psi * dx_f
a_f = a_f0 + a_f1
L_f = f_f * psi * dx_f

# --- Lagrange multiplier (constraint tr(p_m) = p_f) ------------------
# Dokken’s example handles this through interface coupling on ds(Gamma_tag)
a_l0 = ufl.avg(p_m) * mu * ds_m
a_l1 = -p_f * mu * dx_f
a_l = a_l0 + a_l1
L_l = fem.Constant(msh, 0.0) * mu * dx_f

# ---------------------------------------------------------------------
# 6. Combine interface weak form
# ---------------------------------------------------------------------
a_gamma = a_f + a_l
L_gamma = L_f + L_l

# ---------------------------------------------------------------------
# Print symbolic forms (optional sanity check)
# ---------------------------------------------------------------------
print("a_m =", a_m)
print("a_gamma =", a_gamma)

a_m = { conj(((grad(v_0^0)) : (grad(v_1^0)))) } * dx(<Mesh #1>[everywhere], {})
  +  { v_0^0 * -1 * v_1^2 } * ds(<Mesh #1>[2], {})
a_gamma = { conj(((grad(v_0^1)) : (grad(v_1^1)))) } * dx(<Mesh #2>[everywhere], {})
  +  { v_0^1 * v_1^2 } * dx(<Mesh #2>[everywhere], {})
  +  { v_0^2 * -1 * v_1^1 } * dx(<Mesh #2>[everywhere], {})
  +  { v_0^2 * 0.5 * (((v_1^0)(+)) + ((v_1^0)(-))) } * ds(<Mesh #1>[2], {})


In [10]:
a = a_m + a_gamma
L = L_m + L_gamma

In [11]:
# Compute bounding box to locate sides
coords = omega.geometry.x
x = coords[:, 0]
y = coords[:, 1]

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

# Tolerance for side detection
tol = 1e-10 * max(xmax - xmin, ymax - ymin)

# Locate dofs on each side
left_dofs   = fem.locate_dofs_geometrical(V_m, lambda x: np.isclose(x[0], xmin, atol=tol))
right_dofs  = fem.locate_dofs_geometrical(V_m, lambda x: np.isclose(x[0], xmax, atol=tol))
bottom_dofs = fem.locate_dofs_geometrical(V_m, lambda x: np.isclose(x[1], ymin, atol=tol))
top_dofs    = fem.locate_dofs_geometrical(V_m, lambda x: np.isclose(x[1], ymax, atol=tol))

# Union of all boundary dofs for which we enforce a value
all_dofs = np.unique(np.concatenate([left_dofs, right_dofs, bottom_dofs, top_dofs]))

# Build a Function-valued BC so we can assign side-dependent values
p_m_bc = fem.Function(V_m)
p_m_bc.x.array[:] = 0.0  # default 0 (bottom + right)
p_m_bc.x.array[left_dofs] = 1.0
p_m_bc.x.array[top_dofs]  = 1.0

# One BC object over the union of dofs with piecewise values
bc_pm = fem.dirichletbc(p_m_bc, all_dofs)

bcs = [bc_pm]

In [13]:
from dolfinx.fem import petsc
entity_maps = [gamma_to_omega]
problem = petsc.LinearProblem(
    a, L,
    bcs=bcs,
    entity_maps=entity_maps,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",        # or "superlu_dist"
        "ksp_error_if_not_converged": True,
    },
    petsc_options_prefix="pmix_"
)

U = problem.solve()          # U is a Function in W
p_m_sol, p_f_sol, lmbd_sol = U.split()

ValueError: too many values to unpack (expected 1)